In [4]:
import pandas as pd


df = pd.read_csv('Data/Ridership.csv')
df.sample(5)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Ridership.csv'

In [ ]:
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4,
          'May': 5, 'June': 6, 'July': 7, 'August': 8,
          'September': 9, 'October': 10, 'November': 11, 'December': 12}

df['month'] = df['Month'].map(months)

df['date'] = pd.to_datetime(df[['Year', 'month', 'Day']])
df.head()

,Year,Month,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19,month_num,month,date
0,2019,January,1,1,Corridor_1,n,Station_1,Weekend/Holiday,174,3,0,1,1,2019-01-01
1,2019,January,1,1,Corridor_1,n,Station_2,Weekend/Holiday,378,10,0,1,1,2019-01-01
2,2019,January,1,1,Corridor_1,n,Station_3,Weekend/Holiday,599,12,0,1,1,2019-01-01
3,2019,January,1,1,Corridor_2,n,Station_4,Weekend/Holiday,2759,35,0,1,1,2019-01-01
4,2019,January,1,1,Corridor_2,n,Station_5,Weekend/Holiday,2629,36,0,1,1,2019-01-01


In [ ]:
df.to_parquet('Data/parquetdata/', index=False, engine='pyarrow', compression='snappy', partition_cols=['Year', 'Month'])

In [ ]:
pd.read_parquet('Data/Ridership.parquet', engine='pyarrow', columns=['Year', 'Month']).head()

,Year,Month
0,2019,January
1,2019,January
2,2019,January
3,2019,January
4,2019,January


In [ ]:
a = []
b = [1, 2, 3]

a.append(b)

a

[[1, 2, 3]]

In [ ]:
a.extend(b)

a

[[1, 2, 3], 1, 2, 3, 1, 2, 3]

In [ ]:
file_path = glob(f'Data/parquetdata/Year=2019/Month=August/*.parquet')
df = pd.read_parquet(file_path[0], engine='pyarrow')
df.sample(5)

,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19,month_num,month,date
254,7,32,Corridor_2,y,Station_3,PM Peak,9808,12,0,8,8,2019-08-07
18,1,31,Corridor_4,y,Station_3,AM Peak,49,1,0,8,8,2019-08-01
964,22,34,Corridor_1,y,Station_3,Evening,1711,5,0,8,8,2019-08-22
953,21,34,Corridor_7,y,Station_21,Midday,1958,4,0,8,8,2019-08-21
1214,27,35,Corridor_3,y,Station_15,AM Peak,719,1,0,8,8,2019-08-27


In [ ]:
from glob import glob

file_paths = []
for month in ['August', 'April']:
    file_paths.extend(glob(f'Data/parquetdata/Year=2019/Month={month}/*.parquet'))


file_paths

['Data/parquetdata/Year=2019/Month=August/e4f0ecc145844b3782fc7dd6869bd8ca-0.parquet',
 'Data/parquetdata/Year=2019/Month=April/e4f0ecc145844b3782fc7dd6869bd8ca-0.parquet']

In [ ]:
dfs = []

for file_path in file_paths:
    df = pd.read_parquet(file_path, engine='pyarrow', columns=['date', 'Period', 'Ridership'])
    dfs.append(df)

partial_df1 = pd.concat(dfs, ignore_index=True)
partial_df1.sample(10)

,date,Period,Ridership
2421,2019-04-22,Midday,1443
238,2019-08-07,AM Peak,271
2343,2019-04-18,AM Peak,2925
2659,2019-04-26,Evening,5010
2273,2019-04-17,Midday,1715
1447,2019-04-01,PM Peak,5263
2784,2019-04-30,AM Peak,1183
1828,2019-04-08,Evening,1341
2401,2019-04-22,AM Peak,610
665,2019-08-15,PM Peak,308


In [ ]:
df.columns.to_list()

['Day',
 'Week Number',
 'Corridor',
 'Workday',
 'Station',
 'Period',
 'Ridership',
 'N_trains',
 'Covid19',
 'month_num',
 'month',
 'date']

In [ ]:
dfs = []

for file_path in file_paths:
    df = pd.read_parquet(file_path, engine='pyarrow', columns=['Day', 'Week Number', 'Corridor', 'Station', 'date', 'Period'])
    dfs.append(df)

partial_df2 = pd.concat(dfs, ignore_index=True)
partial_df2.sample(10)

,Day,Week Number,Corridor,Station,date,Period
323,8,32,Corridor_3,Station_16,2019-08-08,Midday
614,14,33,Corridor_6,Station_3,2019-08-14,PM Peak
2514,24,17,Corridor_4,Station_8,2019-04-24,AM Peak
1927,10,15,Corridor_3,Station_4,2019-04-10,Evening
508,13,33,Corridor_4,Station_9,2019-08-13,Midday
676,15,33,Corridor_6,Station_20,2019-08-15,Midday
1753,6,14,Corridor_1,Station_1,2019-04-06,Weekend/Holiday
1463,1,14,Corridor_2,Station_14,2019-04-01,AM Peak
2268,17,16,Corridor_3,Station_13,2019-04-17,AM Peak
668,15,33,Corridor_3,Station_3,2019-08-15,Midday


In [ ]:
final_df = partial_df1.merge(partial_df2, on=['date', 'Period'])
final_df.head()

,date,Period,Ridership,Day,Week Number,Corridor,Station
0,2019-08-01,AM Peak,4716,1,31,Corridor_1,Station_1
1,2019-08-01,AM Peak,4716,1,31,Corridor_1,Station_6
2,2019-08-01,AM Peak,4716,1,31,Corridor_4,Station_7
3,2019-08-01,AM Peak,4716,1,31,Corridor_4,Station_8
4,2019-08-01,AM Peak,4716,1,31,Corridor_4,Station_9


In [ ]:
final_df['Ridership'].sum()

np.int64(104803526)

In [ ]:
final_df[final_df['Corridor'] == 'Corridor_1']['Ridership'].sum()

NameError: name 'final_df' is not defined